In [19]:
import requests
import util.item_analysis as ui
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

market = ['올가홀푸드']
addr = '서울시 종로구'

# 고객이 입력한 상품을 파는 매장을 리스트로 가져오기
market_list = market

In [20]:
# 고객이 입력한 주소 '시', '구'만 가져오기 (이외의 정보는 지도 검색 시 필요없음으로 확인)
addr_list = addr.split(' ')
addr_list2 = []
for i in addr_list:
    if i[-1] == '시':
        addr_list2.append(i)
    elif i[-1] == '구':
        addr_list2.append(i)
addr = ' '.join(addr_list2)
addr


'서울시 종로구'

In [21]:
# 고객이 입력한 주소 좌표 구하기
with open('keys/카카오api.txt') as file:
    kakao_key = file.read()

base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}
url = f'{base_url}?query={quote(addr)}'
result = requests.get(url, headers=header).json()
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])

In [22]:
lat, lng

(37.5735042429813, 126.978989954189)

In [23]:
# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)

In [24]:
# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

In [25]:
# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)

In [88]:
# '상세 주소 화살표' 클릭
juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
juso_way[0].click()
time.sleep(1)


In [26]:
# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source)
market_list = soup.find('ul', class_='eDFz9')
mart = market_list.find('li', recursive=False)
mart

<li class="VLTHu" data-laim-exp-id="undefined"><div class="qbGlu"><div class="ouxiq"><a class="P7gyV" href="https://m.place.naver.com/place/31663641?entry=ple" role="button"><div class="ApCpt"><div class="C6RjW"><span class="place_bluelink YwYLL">올가 롯데백화점본점</span><span class="YzBgS">유기농산물마트</span></div></div></a><a class="P7gyV" href="https://m.place.naver.com/place/31663641?entry=ple" role="button"></a><div class="d7iiF"><div class="KgfA6"><span class="lWwyx NVngW"><span class="place_blind">현재 위치에서</span>1.0km</span><span class="lWwyx"><a aria-expanded="false" aria-haspopup="true" class="uFxr1" href="#" role="button"><span class="Pb4bU">서울 중구 소공동</span><span class="_44_8"><span class="place_blind">상세주소 열기</span><svg aria-hidden="true" class="Y2sSu" viewbox="0 0 13 8" xmlns="http://www.w3.org/2000/svg"><path d="M6.5 7.7L.1 1.4.8.7l5.6 5.6L12 .7l.7.7-6.2 6.3z"></path></svg></span></a></span></div></div></div></div><div class="B5sp_" data-nclicks-area-code="vsl"><div class="kXw6i"><div><

In [27]:
results = []
# 현재 위치에서 마켓 거리
m_distance = mart.select_one('span.lWwyx.NVngW').get_text().split('서')[1]
# 마켓 이름
m_title = mart.select_one('span.place_bluelink.YwYLL').get_text()
# 마켓 도로명 주소
m_addr = mart.select_one('div.zZfO1').get_text()[3:-2]
# 딕셔너리 형태로 저장
market_data = {
    'm_distance': m_distance,
    'm_title': m_title,
    'm_addr': m_addr
}
# 딕셔너리 형태로 저장한 것을 리스트에 저장 (3가지 마켓을 넣어야 하므로)
results.append(market_data)
results

AttributeError: 'NoneType' object has no attribute 'get_text'